In [1]:

from sklearn.cluster import MiniBatchKMeans
import numpy as np

In [14]:
#load modules
import datacube
from datacube.storage import masking
from datacube import Datacube
from datetime import datetime
from skimage import exposure

import rasterio
from datacube_stats.statistics import GeoMedian
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import geopandas as gpd
from sklearn.ensemble import ExtraTreesClassifier
import datacube_stats
import os
# Replace '156' and 'ck9738' with the path to your own home directory on the VDI
dc = Datacube(config='/home/547/ck9738/datacube-s2.conf')

In [15]:
#load sentinal data
sensors= ['s2a_ard_granule'] #pick the sentinal satelites you want

bands_of_int =['red',
        'blue', 'green', 'pixel_quality'
        ] #pick the sentinal bands that you want, here i am just using visible light    

query = {
        'lat': (-33.107410, -33.216838),
        'lon': (148.264306, 148.470191),
        'output_crs': 'EPSG:3577',
        'resolution': (-10, 10),
        'time':('2016-01-01', '2016-07-30')
        }
      
# use s2b_ard_granule 	 for S2B
data_sent = dc.load(product='s2a_ard_granule', measurements=bands_of_int, group_by='solar_day', **query)
clear_pixels = np.logical_and(data_sent.pixel_quality != 2, data_sent.pixel_quality != 3)

data_sent = data_sent.where(clear_pixels)


/g/data/v10/public/modules/dea-env/20190329/lib/python3.6/site-packages/xarray/core/variable.py:134: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  return np.asarray(pd.Series(values.ravel())).reshape(values.shape)
Error opening source dataset: /g/data/dz56/datacube/002/S2_MSI_ARD/packaged/30S145E-35S150E/S2A_OPER_MSI_ARD_TL_SGS__20160114T084618_A002930_T55HFD_N02.01/NBAR/S2A_OPER_MSI_ARD_TL_SGS__20160114T084618_A002930_T55HFD_NBAR_B04.TIF


RasterioIOError: /g/data/dz56/datacube/002/S2_MSI_ARD/packaged/30S145E-35S150E/S2A_OPER_MSI_ARD_TL_SGS__20160114T084618_A002930_T55HFD_N02.01/NBAR/S2A_OPER_MSI_ARD_TL_SGS__20160114T084618_A002930_T55HFD_NBAR_B04.TIF: No such file or directory

In [5]:
X = np.array([[1, 2], [1, 4], [1, 0],
              [4, 2], [4, 0], [4, 4],
              [4, 5], [0, 1], [2, 2],
              [3, 2], [5, 5], [1, -1]])
X

array([[ 1,  2],
       [ 1,  4],
       [ 1,  0],
       [ 4,  2],
       [ 4,  0],
       [ 4,  4],
       [ 4,  5],
       [ 0,  1],
       [ 2,  2],
       [ 3,  2],
       [ 5,  5],
       [ 1, -1]])

In [6]:
# manually fit on batches
kmeans = MiniBatchKMeans(n_clusters=2,
                         random_state=0,
                         batch_size=6)
kmeans = kmeans.partial_fit(X[0:6,:])
kmeans = kmeans.partial_fit(X[6:12,:])
kmeans.cluster_centers_

array([[1, 1],
       [3, 4]])

In [7]:
kmeans.predict([[0, 0], [4, 4]])

array([0, 1], dtype=int32)

In [8]:
# fit on the whole data
kmeans = MiniBatchKMeans(n_clusters=2,
                         random_state=0,
                         batch_size=6,
                         max_iter=10).fit(X)
kmeans.cluster_centers_

array([[3.95918367, 2.40816327],
       [1.12195122, 1.3902439 ]])

In [9]:
kmeans.predict([[0, 0], [4, 4]])

array([1, 0], dtype=int32)